In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from uszipcode import ZipcodeSearchEngine
from geopy.geocoders import Nominatim

import pandas as pd
import seaborn.apionly as sns
from datetime import date, datetime
from haversine import haversine

# statistics package
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

# packages for mapping
from mpl_toolkits.basemap import Basemap

# packages for interactive graphs
from ipywidgets import widgets, interact
from IPython.display import display
from copy import deepcopy as copy
import time
from geopy.geocoders import Nominatim
from time import sleep
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
historical_data = pd.read_csv('clean_data.csv')

In [4]:
historical_data.head()

,Unnamed: 0,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,...,dropoff_hour,pickup_date,dropoff_date,distance,speed,day_interval,waiting_time,waiting_penalty,pickup_zipcode,dropoff_zipcode
0,186987,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 20:38:00,2013-01-13 20:58:00,1,1200,...,20,2013-01-13,2013-01-13,10.418191,31.254572,2,0.000000,0.000000,11371,10025
1,136205,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:08:00,2013-01-13 21:10:00,1,120,...,21,2013-01-13,2013-01-13,0.376856,11.305694,2,21.362104,0.029670,10023,10024
2,114412,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:31:00,2013-01-13 21:35:00,1,240,...,21,2013-01-13,2013-01-13,0.996570,14.948549,2,497.832494,0.691434,10016,10168
3,148407,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:39:00,2013-01-13 21:46:00,1,420,...,21,2013-01-13,2013-01-13,4.608048,39.497556,2,1872.055354,2.600077,10010,10278
4,114616,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 22:12:00,2013-01-13 22:15:00,1,180,...,22,2013-01-13,2013-01-13,1.231572,24.631431,2,3194.887668,4.437344,10003,10010


## Reinforcement Model

In [5]:
class TaxiWorld():
    def __init__(self):
        self.waiting_penalty = 5.00
        pass
                
    def getReward(self, row):
        return row['total_amount'] - row['waiting_time']/3600*self.waiting_penalty
    
    def set_waiting_penalty(self, penalty):
        self.waiting_penalty = penalty
        
    def getState(self):
        print('Current State')
        return self.s
    
    def setState(self, s):
        self.s = s
        
    def getStateSize(self, trip_and_fare):
        return len(trip_and_fare['zip_code'].unique())
    
    def getZipCodes(self, trip_and_fare):
        return self.zip_codes
    
    def getActionSize(self):
        return len(self.actions)

    def nextAction(self, s):
        print('Calculate Next Action based on state')
        
    def check_Q_table(self, s):
        if s not in self.Q:
            self.Q[s] = dict((action, 0.0) for action in self.actions)
            


In [6]:
env = TaxiWorld()

In [7]:
class TaxiRevenue:
    def __init__(self, env, penalty=5.0):
        self.env = env
        self.env.set_waiting_penalty(penalty)
        self.Q = dict()
    
    
    def generate_key(self, row):
        return str(row['pickup_zipcode']) + '_'  + str(row['pickup_weekday']) + '_' + str(row['day_interval'])
    
    def get_tuples(self, key):
        tuples = key.split('_')
        return {'pickup_zipcode': tuples[0],
                   'pickup_weekday': tuples[1],
                   'day_interval': tuples[2]}
    
    def build_Q_table(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            reward = self.env.getReward(row)
            if key not in self.Q:
                self.Q[key] = [reward,1]
            else:
                self.Q[key] = [self.Q[key][0] +reward, self.Q[key][1] + 1]
                
    def greedy(self, s):
        return np.argmax(self.Q[s[0]]) 

    def epsilon_greed(self, epsilon, s):
        if np.random.rand() < epsilon:
            return np.random.randint(self.n_a)
        else:
            return self.greedy(s)
        
    def train(self, trip_and_fare):
        self.build_Q_table(trip_and_fare)
        return self.Q
        
    def test(self):
        return self.env.getState()

In [8]:
taxiRevenue = TaxiRevenue(env)
_Q = taxiRevenue.train(sorted_train_data)

NameError: name 'sorted_train_data' is not defined